<a href="https://colab.research.google.com/github/johyunkang/python-ml-guide/blob/main/python_ml_perfect_guide_08_TextAnal_20NewsGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 04 텍스트 분류 실습 - 20 뉴스그룹 분류
참조 : https://github.com/wikibook/pymldg-rev/tree/master/8%EC%9E%A5

- 희소행렬 분류를 효과적으로 잘 처리할 수 있는 알고리즘
    - 로지스틱 회귀
    - SVM
    - 나이브 베이즈

#### 텍스트 정규화


In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [3]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [4]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
# 개별 데이터가 텍스트로 어떻게 구성되어 있는지 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [6]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
x_train = train_news.data
y_train = train_news.target

# subset='test'로 테스트 데이터만 추출, remove=('headers', 'footers', 'quotest')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)
x_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))


학습 데이터 크기 11314, 테스트 데이터 크기 7532


#### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorization으로 피처 벡터화 변환 수행
cv = CountVectorizer()
cv.fit(x_train)
x_train_cv = cv.transform(x_train)

# 학습데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
x_test_cv = cv.transform(x_test)

print('학습 데이터 텍스트의 CountVectorizer shape:', x_train_cv.shape)

학습 데이터 텍스트의 CountVectorizer shape: (11314, 101631)


- 11314 개의 문서에서 피처, 즉 단어가 101631개 만들어짐
- 피처 벡터화된 데이터에 로지스틱 회귀 적용해 뉴스 그룹에 대한 분류 예측해 보겠음

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습 / 예측 / 평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(x_train_cv, y_train)
pred = lr_clf.predict(x_test_cv)
print('CountVectorized LogisticRegression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized LogisticRegression의 예측 정확도는 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


- Count 기반 벡터화 적용된 데이터 세트의 로지스틱 회귀의 예측 정확도는 약 0.608이다.
- 이번에는 TF-IDF 기반의 벡터화를 변경해 예측 모델을 수행

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(x_train)
x_train_tfidf_vect = tfidf_vect.transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

# LR을 이용해 학습 / 예측 / 평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(x_train_tfidf_vect, y_train)
pred = lr_clf.predict(x_test_tfidf_vect)
print('TF-IDF LogisticRegression의 예측정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF LogisticRegression의 예측정확도는 0.674


- TF-IDF의 예측 정확도는 약 0.674 이다.
- TF-IDF가 단순 카운트 기반보다 훨신 높은 예측 정확도를 제공
- 다양한 파라미터를 적용해 TF-IDF 수행

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1, 1)에서 (1, 2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)
tfidf_vect.fit(x_train)
x_train_tfidf_vect = tfidf_vect.transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

lr_clf = LogisticRegression()
lr_clf.fit(x_train_tfidf_vect, y_train)
pred = lr_clf.predict(x_test_tfidf_vect)
print('파라미터 추가된 TF-IDF LogisticRegression의 예측정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

파라미터 추가된 TF-IDF LogisticRegression의 예측정확도는 0.692


- 파라미터가 추가된 TF-IDF의 예측 정확도는 약 0.692
- 이번에는 GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3폴드 세트로 설정
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(x_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


- 로지스틱 회귀의 C가 10 일 때 GridSearchCV의 교차검증 테스트 세트에서 가장 좋은 예측 성능을 나타냄
- 이를 데이터 세트에 적용해 약 0.703으로 이전보다 약간 향상된 성능 수치가 됨

#### 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합

- 위의 텍스트 분류 예제를 Pipeline을 이용해 다시 작성한 코드

In [11]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요 없음
# pipeline의 fit()과 predict() 만으로 한 꺼번에 피처 벡터화와 ML 학습 / 예측이 가능
pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


- GridSearchCV에 Estimator가 아닌 Pipeline을 입력할 경우 param_grid의 입력 값 설정이 기존과 약간 다름
- key 값을 살펴보면 'tfidf_vect__ngram_range'와 같이 하이퍼 파라미터명이 객체 변수명과 결합돼 제공됨
- 가령 TfdifVectorizer 객체 변수인 tfdif_vect의 ngram_range 파라미터 값을 변화시키면서 최적화하기를 원한다면 객체 변수명인 tfdif_vect에 언더바('_') 2개를 연달아 붙인 뒤 파라미터명인 ngram_range를 결합해 'tfdif_vect__ngram_range'를 key 값으로 할당
- Pipeline + GridSearchCV 유의점은 최적화 시 너무 많은 튜닝 시간이 소모됨

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용됨
# 파라미터  / 하이퍼 파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
          'tfidf_vect__max_df':[100, 300, 700],
          'lr_clf__C': [1, 5, 0]
          }

# GridSearchCV의 생성자에 Estimator 가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(x_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(x_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


- TfidfVectorizer 객체의 max_df 가 700, ngram_range 파라미터가 (1, 2)로 피처 벡터화된 데이터 세트에 LogisticRegression의 C 하이퍼 파라미터에 10을 적용해 예측 분류를 수행할 때 가장 좋은 검증 세트 성능 수치가 도출
- 아쉽게도 이렇게 최적화된 파라미터 기반으로 테스트 데이터 세트에 대해 예측했을 때의 정확도는 약 0.702로 크게 개선되지 않음

- **로지스틱 회귀 외에 서포트벡터머신과 나이브베이즈 알고리즘도 희소 행렬 기반의 텍스트 분류에 자주 사용되는 머신러닝 알고리즘임**
- 이들을 이용한 모델도 만들어 볼 것을 권장